In [1]:
# -*- coding: utf-8 -*-
import json
import os
import random
import numpy as np
import math
import torch
from torch import nn
from torch.nn import init
from torch import optim
import time
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import random
from arglex import Classifier

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gzip
import gensim
import logging
import os
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import itertools

arglex = Classifier()

with open("users.json", "r") as f:
    users = json.load(f)

number_of_users = len(users)
print("Finish User loading, with number of users:", number_of_users)
# print(users.keys())
# print(users["ahuggies30"])
get_big_issues = users["ahuggies30"]["big_issues_dict"].keys()

big_issues_list = []

for item in get_big_issues:
    big_issues_list.append(item)

Finish User loading, with number of users: 45348


In [2]:
# ############################################
# # Load Word Embedding
# ############################################


# def sent_shuffle(sent):
#     all_sents = []
#     total_length = len(sent)

#     for i in range(10):
#         random.shuffle(sent)
#         all_sents.append(sent[:])

#     return all_sents


# # sent = ["I", "am", "what", "you", "don't", "expect"]
# # all_sents = sent_shuffle(sent)
# # print()
# # print(all_sents)
# # breakpoint()

# with open("users.json", "r") as f:
#     users = json.load(f)

# sentences = []
# # useful_aspects = [ 'education', 'ethnicity', 'income', 'interested', 'looking', 'number_of_friends',  'party', 'political_ideology', 'president', 'relationship', 'religious_ideology', 'win_ratio', 'gender']
# # useful_aspects = ['political_ideology', 'education', 'ethnicity', 'interested', 'gender',"religious_ideology"]
# useful_aspects = ['political_ideology','education', 'gender', "religious_ideology", "party", "ethnicity"]

# for user in users.values():
#     user_sent = []

#     for aspect in useful_aspects:

#         if user[aspect] == "Not Saying":
#             continue
#         else:
#             # user_sent.append(aspect + ":" + user[aspect])
#             user_sent.append(user[aspect])

#     for key in user["big_issues_dict"].keys():

#         for choice in ["Pro", "Con"]: # , "Und", "N/O"]:
#             this_sent = user_sent
#             if user["big_issues_dict"][key] == choice:
#                 this_sent.append(key + "-" + choice)
#                 if len(this_sent) > 0:
#                     sent_list = this_sent
#                     if len(sent_list) > 0:
#                         sentences.extend(sent_list)


# print("Total number of sentences generated", len(sentences))

# language_model = Word2Vec(sentences, size=8, window=20, sg=0)
# # language_model = Word2Vec.load("user_aspect_embedding.bin")
# print("Finish loading the word vector")

In [4]:

# language_model.wv.save("user_ascpect_embedding.bin")

language_model = Word2Vec.load("user_aspect_embedding.bin")

In [25]:
# Helper Function

def target_counter(target_list, users):
    target_counters = dict()
    for target in target_list:
        target_counters[target] = dict()

    for user in users.values():
        for target in target_list:
            if not target_counters[target].get(user[target]):
                if "Christian" in  user[target]:
                    user[target] = "Christian"
                target_counters[target][user[target]] = 1
            else:
                if "Christian" in  user[target]:
                    user[target] = "Christian"
                target_counters[target][user[target]] += 1

    return target_counters


def criterion_generator(target_list, target_counters, bar):
    criterion = dict()

    for target in target_list:
        criterion[target] = []
        for item in target_counters[target].keys():
            if item == "Not Saying":
                continue
            if target_counters[target][item] > bar:
                criterion[target].append(item)
        if len(criterion[target]) == 0:
            del criterion[target]

    return criterion

religious_ideology = "religious_ideology"  # Christian or Atheist
political_ideology = "political_ideology"  # Liberal or Conservative


def user_filter(users, target_list, bar):
    target_counters = target_counter(target_list, users)
    criterion = criterion_generator(target_list, target_counters, bar)

    valid_users = []
    for user in users.values():
        indicator = 1

        for key in criterion.keys():
            if not user.get(key):
                indicator = 0
                break
            elif user.get(key) not in criterion[key]:
                indicator = 0
                break

        if indicator:
            valid_users.append(user)

    # print("Number of valid users:", len(valid_users))
    return valid_users, criterion

number = 0


def get_user_aspect_vec(user, criterion, model, target_big_issues):
    user_aspect_vec = []
    for key in criterion.keys():
        init = [0.0 for i in range(len(criterion[key]))]

        if user[key] in criterion[key]:
            index = criterion[key].index(user[key])
            init[index] = 1.0
        
        # one-hot
        user_aspect_vec.extend(init)

        # Embedding
        w2v_vec = model.wv[user[key]]
        user_aspect_vec.extend(w2v_vec)

        # Similarity
        sim_pro_sum = 0
        sim_con_sum = 0
        sim_pro_mul = 0
        sim_con_mul = 0

        for target_big_issue in target_big_issues:
            # choice = user["big_issues_dict"][target_big_issue]
            try:
                # w2v_vec_issue = model.wv[target_big_issue + "-" + choice]
                choice1 = "Pro"
                choice2 = "Con"
                sim_pro = model.wv.similarity(user[key], target_big_issue + "-" + choice1)
                sim_con = model.wv.similarity(user[key], target_big_issue + "-" + choice2)
                # user_aspect_vec.append(sim_pro)
                # user_aspect_vec.append(sim_con)
                # user_aspect_vec.append(sim_pro/sim_con)
                # user_aspect_vec.append(sim_pro - sim_con)
                sim_pro_sum += sim_pro
                sim_con_sum += sim_con
                if sim_pro == 0:
                    sim_pro = 0.0001
                if sim_con == 0:
                    sim_con = 0.0001
                sim_pro_mul *= sim_pro
                sim_con_mul *= sim_con
            except:
                global number
                number += 1
                print("Number of missing in similarity part", number)
                user_aspect_vec.extend(np.array(w2v_vec) - np.array(w2v_vec_issue))

    user_aspect_vec.append(sim_pro_sum)
    user_aspect_vec.append(sim_con_sum)
    user_aspect_vec.append(sim_pro_sum/sim_con_sum)
    user_aspect_vec.append(sim_pro_sum - sim_con_sum)

    return user_aspect_vec


def language_feature_generator(text):

    zero = [0.0, 0.0, 0.0, 0.0]

    if len(text) == 0:
        return zero

    sent_tokenize_list = sent_tokenize(text)

    average_sent_length = 0.0
    average_sent_subjectivity = 0.0
    average_sent_polarity = 0.0

    count = 0
    for sentence in sent_tokenize_list:
        count += 1
        sent_blob = TextBlob(sentence)
        average_sent_subjectivity += sent_blob.sentiment.subjectivity
        average_sent_polarity += sent_blob.sentiment.polarity
        average_sent_length += len(word_tokenize(sentence))

    if count == 0:
        return zero

    average_sent_length = float(average_sent_length / count)
    average_sent_subjectivity = float(average_sent_subjectivity / count)
    average_sent_polarity = float(average_sent_polarity / count)

    return len(sent_tokenize(text)), average_sent_length, average_sent_subjectivity, average_sent_polarity


def get_user_language_vec(user):
    language_vec = []

    if not user.get("opinion_arguments"):
        return [0.0 for i in range(9)]

    arguments = user["opinion_arguments"]
    # number of opinions
    # language_vec.append(float(len(arguments)))

    average_sentence_number = 0.0
    average_sentence_length = 0.0
    average_sentence_sub = 0.0
    average_sentence_polar = 0.0

    opinion_count = 0
    all_text = ""
    
    for opinion in arguments:
        opinion_count += 1
        sent_number, sent_length, sent_sub, sent_polar = language_feature_generator(opinion["opinion text"])
        average_sentence_number += sent_number
        average_sentence_length += sent_length
        average_sentence_sub += sent_sub
        average_sentence_polar += sent_polar
        
        all_text += " "
        all_text += opinion["opinion text"]
    
    average_sentence_number/opinion_count
    average_sentence_length/opinion_count
    average_sentence_sub = average_sentence_sub / opinion_count
    average_sentence_polar = average_sentence_polar / opinion_count

    language_vec.append(average_sentence_number)
    language_vec.append(average_sentence_length)
    language_vec.append(average_sentence_sub)
    language_vec.append(average_sentence_polar)
    
    try:
        all = ['0-Assessments', '1-Authority', '2-Causation', '3-Conditionals', '4-Contrast', '5-Difficulty', '6-Doubt', '7-Emphasis',\
         '8-Generalization', '9-Inconsistency', '10-Inyourshoes', '11-Necessity', '12-Possibility', '13-Priority', '14-Rhetoricalquestion',\
         '15-Structure', '16-Wants']
        lexicon_score = arglex.analyse(all_text)
        useful_lex = [lexicon_score[1], lexicon_score[3], lexicon_score[4], lexicon_score[5], lexicon_score[11]]
        language_vec.extend(useful_lex)
        # print(useful_lex)
    except:
        language_vec.extend([0.0 for i in range(5)])
    
    
    while len(language_vec) != 9:
# #         print(user["opinion_arguments"])
# #         print(useful_lex)
# #         print(language_vec)
        language_vec.append(0.0)
    
    return language_vec

In [18]:



target_big_issue = ["Drug Legalization", "Medical Marijuana"]#,"Gay Marriage"]

my_user = users["ahuggies30"]
my_user = users["yomama12"]
# print("Example of user aspect vector: ", get_user_aspect_vec(my_user, criterion, language_model, target_big_issue))
print("Example of user language vector", lenget_user_language_vec(my_user))

Example of user language vector [0.0, 0.0, 0.0, 0.0, 0.14286, 14.0, 55.63333333333333, 0.24047138047138047, -0.01085858585858585]


In [20]:
def get_training_set(users, target_big_issue, big_issue_list, criterion, w2v_model):
    xTr = []
    yTr = []
    # possible = ["Pro", "Con", "N/O", "N/S", "Und"]
    possible = ["Pro", "Con", "N/O", "Und"]
    see = dict()
    for i in possible:
        see[i] = 0

    for user in users:
        vec = get_user_aspect_vec(user, criterion, w2v_model, target_big_issue)
        label = []
        for target in target_big_issue:
            if user["big_issues_dict"][target] == "N/S":
                continue
            elif user["big_issues_dict"][target] == "N/O" or user["big_issues_dict"][target] == "Und":
                continue
                label.extend([0.0, 1.0])
            else:
                # mask = [0.0, 0.0, 0.0, 0.0]
                mask = [0.0, 0.0]
                mask[possible.index(user["big_issues_dict"][target])] = 1.0
                label.extend(mask)

        # see[possible[label]] += 1

        vec.extend(get_user_language_vec(user))

        if vec and len(label) == 2 * len(target_big_issue):
            # no N/S
            xTr.append(vec)
            yTr.append(label)

    return xTr, yTr


class model(nn.Module):

    def __init__(self, vocab_size, hidden_dim=64, out_dim=2, output_size=3):
        super(model, self).__init__()
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.embeds = nn.Linear(vocab_size, hidden_dim)
        self.encoder = nn.Linear(hidden_dim, hidden_dim)

        self.out = nn.Linear(hidden_dim, out_dim * output_size)

        self.loss = nn.BCELoss(reduction='mean')
        # self.loss = nn.MultiLabelMarginLoss()
        

        self.output_size = output_size
        self.indicator = 1

    def compute_loss(self, pred_vec, gold_seq):
        
#         criterion = nn.CrossEntropyLoss()
#         loss = 0
#         for lx in range(len(pred_vec))
        
        return self.loss(pred_vec, gold_seq)

    def forward(self, input_vectors):
        #         input_vectors = self.embeds(torch.tensor(input_seq))

        input_vectors = self.embeds(input_vectors)
        # hidden = input_vectors
        # input_vectors = input_vectors.unsqueeze(1)
        # _, hidden = self.encoder(input_vectors)
        # print(output.size())
        input_vectors = torch.nn.functional.leaky_relu(input_vectors, negative_slope=0.01, inplace=False)
        hidden = self.encoder(input_vectors)
        hidden = torch.nn.functional.leaky_relu(hidden, negative_slope=0.01, inplace=False)
        hidden = self.encoder(hidden)
        output = torch.nn.functional.leaky_relu(hidden, negative_slope=0.01, inplace=False)
        output = self.out(output)
        predictions = torch.sigmoid(output)
        predictions = predictions.squeeze()

        # print("predictions:", predictions)
        idxs = []
        for i in range(self.output_size):
            prediction = predictions[(i * self.out_dim):((i + 1)) * self.out_dim]
            prediction = prediction.squeeze()
            val, idx = torch.max(prediction, 0)
            # idx = idx.item
            if idx == 0:
                # idxs.extend([1.0, 0.0, 0.0, 0.0])
                idxs.extend([1.0, 0.0])
            elif idx == 1:
                idxs.extend([0.0, 1.0])
                # idxs.extend([0.0, 1.0, 0.0, 0.0])
            elif idx == 2:
                idxs.extend([0.0, 0.0, 1.0])
            elif idx == 3:
                idxs.extend([0.0, 0.0, 0.0, 1.0])

        return predictions, idxs

In [9]:
################### Find the nearest via Embedding ##########################
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

sentences = []

for user in users.values():
    user_pro = []
    user_con = []
    user_und = []
    user_no = []
    # print(user["big_issues_dict"])
    for key in user["big_issues_dict"].keys():
        if user["big_issues_dict"][key] == "Pro":
            user_pro.append(key)
        if user["big_issues_dict"][key] == "Con":
            user_con.append(key)
        if user["big_issues_dict"][key] == "Und":
            user_und.append(key)
        if user["big_issues_dict"][key] == "N/O":
            user_no.append(key)

    if len(user_pro) > 0:
        sentences.append(user_pro)
    if len(user_con) > 0:
        sentences.append(user_con)
    if len(user_und) > 0:
        sentences.append(user_und)
    if len(user_no) > 0:
        sentences.append(user_no)

print(len(sentences))

bi_model = Word2Vec(sentences, size=5, window=50)
print(bi_model)

53905
Word2Vec(vocab=48, size=5, alpha=0.025)


In [10]:
words = list(bi_model.wv.vocab)
embedding_results = dict()

for word in words:
    # print(word)
    po_list = bi_model.most_similar(positive=[word])
    # print(type(list))
    # print(model.most_similar(positive=[word]))
    
    prior_list = []
    for item in po_list:
        prior_list.append(item[0])
        
    embedding_results[word] = prior_list
    
# print(embedding_results)

C:\Users\xzhao\Anaconda3\envs\4901\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


{'Abortion': ['Drug Legalization', 'National Health Care', 'Civil Unions', 'Global Warming Exists', 'United Nations', 'Welfare', 'Environmental Protection', 'Minimum Wage', 'Social Security', 'Smoking Ban'], 'Animal Rights': ['Euthanasia', 'Legalized Prostitution', 'Medical Marijuana', 'Environmental Protection', 'Global Warming Exists', 'Medicaid & Medicare', 'Minimum Wage', 'Civil Unions', 'Drug Legalization', 'Labor Union'], 'Barack Obama': ['Socialism', 'Globalization', 'Estate Tax', 'Free Trade', 'National Retail Sales Tax', 'Affirmative Action', 'Progressive Tax', 'Social Programs', 'Euthanasia', 'Term Limits'], 'Capitalism': ['Free Trade', 'Homeschooling', 'Globalization', 'Barack Obama', 'Gold Standard', 'Legalized Prostitution', 'Socialism', 'Electoral College', 'Affirmative Action', 'Federal Reserve'], 'Environmental Protection': ['Global Warming Exists', 'Medical Marijuana', 'Drug Legalization', 'Animal Rights', 'Minimum Wage', 'Medicaid & Medicare', 'United Nations', 'Civil

In [11]:
print(embedding_results["Abortion"])

this = np.copy(embedding_results["Abortion"])
np.random.shuffle(this)
print(this)

print(embedding_results["Abortion"])

['Drug Legalization', 'National Health Care', 'Civil Unions', 'Global Warming Exists', 'United Nations', 'Welfare', 'Environmental Protection', 'Minimum Wage', 'Social Security', 'Smoking Ban']
['Civil Unions' 'Drug Legalization' 'Global Warming Exists'
 'United Nations' 'Minimum Wage' 'Welfare' 'Environmental Protection'
 'Social Security' 'National Health Care' 'Smoking Ban']
['Drug Legalization', 'National Health Care', 'Civil Unions', 'Global Warming Exists', 'United Nations', 'Welfare', 'Environmental Protection', 'Minimum Wage', 'Social Security', 'Smoking Ban']


In [ ]:
###### coll = []
acc_collection = dict()
list_len = len(big_issues_list)

from random import randint

target_list = ['political_ideology', 'education', 'gender', "religious_ideology"]

# target_list = ['political_ideology','education', 'gender', "religious_ideology", "party", "ethnicity"]
valid_users, criterion = user_filter(users, target_list, 5)


for k in [1, 2 ,3, 4]:
    
    acc_collection[k] = dict()
    
    for i in range(list_len):
    
#         target = big_issues_list[i]
#         print(str(i+1) + "/"+ str(list_len))
#         print(target)
#         target_big_issue = [target]

#         # Use all together
#         temp = big_issues_list
#         temp[0], temp[i] = temp[i], temp[0]
#         target_big_issue = temp
#         print(str(i) + "/"+ str(list_len))
#         print(temp[0])

        # K

        target = big_issues_list[i]
        print(str(i+1) + "/"+ str(list_len))
        print(target)
        target_big_issue = [target]
        
        if k == 1:
            print("single task")
        else:
            aux = embedding_results[target][0:k-1]
            target_big_issue.extend(aux)
            print(aux)
        
        
        # Rand
#         target = big_issues_list[i]
#         print(str(i+1) + "/"+ str(list_len))
#         print(target)
#         target_big_issue = [target]
    
        
#         rand_n = 1
#         temp = np.copy(embedding_results[target])
#         np.random.shuffle(temp)
        
#         aux = temp[0:rand_n]

        
#         target_big_issue.extend(aux)
#         print(aux)   
        
        

        # valid_users, criterion = user_filter(users, target_list, 200)
        unbalanced_xTr, unbalanced_yTr = get_training_set(valid_users, target_big_issue, big_issues_list, criterion, language_model)

        xTr = []
        yTr = []

        number_of_0 = 0
        number_of_1 = 0

        # Find the smaller set
        for i in range(len(unbalanced_xTr)):
            if unbalanced_yTr[i][0] == 0:
                    number_of_0 += 1
            if unbalanced_yTr[i][0] == 1:
                    number_of_1 += 1

        if number_of_0 > number_of_1:
            number_each = number_of_1
        else:
            number_each = number_of_0

        number_of_0 = 0
        number_of_1 = 0

        for i in range(len(unbalanced_xTr)):
            if unbalanced_yTr[i][0] == 0:
                if number_of_0 < number_each:
                    xTr.append(unbalanced_xTr[i])
                    yTr.append(unbalanced_yTr[i])
                    number_of_0 += 1
            if unbalanced_yTr[i][0] == 1:
                if number_of_1 < number_each:
                    xTr.append(unbalanced_xTr[i])
                    yTr.append(unbalanced_yTr[i])
                    number_of_1 += 1

    #     print(len(xTr), len(yTr))
    #     print(number_of_0, number_of_1)


        mask = np.arange(2 * number_each)
        np.random.shuffle(mask)

        temp_xTr = []
        temp_yTr = []

        for pick in mask:
            temp_xTr.append(xTr[pick])
            temp_yTr.append(yTr[pick])

        xTr = temp_xTr
        yTr = temp_yTr

    #     print(len(xTr), len(yTr))
    #     print(len(xTr[0]), len(yTr[0]))


        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        device = torch.device("cpu")

        m = model(len(xTr[0]), 64, 2, len(target_big_issue)).to(device)
        optimizer = optim.SGD(m.parameters(), lr=0.1)

        training_x = xTr[0:int(0.8 * len(xTr))]
        training_y = yTr[0:int(0.8 * len(yTr))]
        test_x = xTr[int(0.8 * len(xTr)):]
        test_y = yTr[int(0.8 * len(yTr)):]
        
        print(len(yTr))

        for epoch in (range(15)):
            m.train()
#             print('training')
            start_train = time.time()
            total_loss = 0

            total_predictions = 0
            correct = 0

            for i in range(int(len(training_x)/10)):
                # print(torch.cuda.current_device())
                predictions = None
                gold_outputs = None
                loss = 0
                optimizer.zero_grad()

                for j in range(10):
                    loss = 0

#                     if torch.cuda.is_available():
#                         gold_output = torch.cuda.FloatTensor(training_y[i + j], device=device)
#                         input_seq = torch.cuda.FloatTensor(training_x[i + j], device=device)
#                     else:
                    gold_output = torch.FloatTensor(training_y[i + j], device=device)
                    input_seq = torch.FloatTensor(training_x[i + j], device=device)

                    prediction_vec, prediction = m(input_seq)

                    total_predictions += 1
                    # gold_outputs = []
                    if prediction[0:2] == training_y[i + j][0:2]:
                        correct += 1
                    if predictions is None:
                        predictions = [prediction_vec]
                        gold_outputs = [gold_output]
                    else:
                        predictions.append(prediction_vec)
                        gold_outputs.append(gold_output)
                    #
                    # for lx in range(len(gold_output)):
                    #     tmp_loss = m.compute_loss(prediction[lx], gold_output[lx])
                    #     loss += tmp_loss

                # print(torch.stack(gold_outputs))
                # print(torch.stack(predictions))
                loss = m.compute_loss(torch.stack(predictions), torch.stack(gold_outputs).squeeze())
                # loss = loss/len(target_big_issue)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            accuracy = correct / total_predictions
#             print("training time: {} for epoch {}".format(time.time() - start_train, epoch))
#             print('total loss:{}'.format(total_loss))
#             print('training accuracy:{}'.format(accuracy))

        m.eval()
        predictions = 0
        correct = 0
        baseline = 0

        max_score = 0
        for i in range(int(len(test_x))):
#             if torch.cuda.is_available():
#                 gold_output = torch.cuda.FloatTensor(test_y[i], device=device)
#                 input_seq = torch.cuda.FloatTensor(test_x[i], device=device)
#             else:
            gold_output = torch.tensor(test_y[i], device=device)
            input_seq = torch.FloatTensor(test_x[i], device=device)

            _, prediction = m(input_seq)

            # print("prediction", prediction)
            # print("test_y", test_y[i])

            if test_y[i][0:2] == prediction[0:2]:
                correct += 1
            if test_y[i][0:2] == [0.0, 1.0]:
                baseline += 1
            predictions += 1

        accuracy = correct / predictions
        
        acc_collection[k][target] = accuracy
        # baseline = baseline / predictions
        assert 0 <= accuracy <= 1
        print('Accuracy:{}'.format(accuracy))
        # print('Baseline:{}'.format(baseline))

    print(acc_collection[k])

1/48
Abortion
single task
4648
Accuracy:0.656989247311828
2/48
Affirmative Action
single task
2294
Accuracy:0.5664488017429193
3/48
Animal Rights
single task
916
Accuracy:0.5380434782608695
4/48
Barack Obama
single task
1788
Accuracy:0.6284916201117319
5/48
Border Fence
single task
3740
Accuracy:0.6203208556149733
6/48
Capitalism
single task
1212
Accuracy:0.5432098765432098
7/48
Civil Unions
single task
1910
Accuracy:0.6047120418848168
8/48
Death Penalty
single task
4594
Accuracy:0.5516866158868335
9/48
Drug Legalization
single task
4164
Accuracy:0.6362545018007203
10/48
Electoral College
single task
1422
Accuracy:0.5473684210526316
11/48
Environmental Protection
single task
1266
Accuracy:0.594488188976378
12/48
Estate Tax
single task
2166
Accuracy:0.543778801843318
13/48
European Union
single task
1264
Accuracy:0.5059288537549407
14/48
Euthanasia
single task
1980
Accuracy:0.5732323232323232
15/48
Federal Reserve
single task
1174
Accuracy:0.5106382978723404
16/48
Flat Tax
single task
2

In [31]:
np_acc = np.array(list(acc_collection[1].values()))
print(np.mean(np_acc))

0.5744383370045895


In [27]:
print(acc_collection)

{1: {'Welfare': 0.5886970172684458}, 2: {'Welfare': 0.6683417085427136}, 3: {'Welfare': 0.6996805111821086}}


In [ ]:
with open('result_with_language.txt', 'w') as outfile:  
    json.dump(acc_collection, outfile)

In [ ]:
# emb_acc = acc_collection
print(acc_collection)
print(emb_acc)
print(all_emb_acc)

In [43]:
# coll = []
# acc_collection = dict()
# list_len = len(big_issues_list)

# target_list = ['political_ideology', 'education', 'gender', "religious_ideology"]

# # target_list = ['political_ideology','education', 'gender', "religious_ideology", "party", "ethnicity"]
# valid_users, criterion = user_filter(users, target_list, 5)

# for k in [1]:
#     for i in range(list_len):
    
# #     target = big_issues_list[i]
# #     print(str(i) + "/"+ str(list_len))
# #     print(target)
# #     target_big_issue = [target]
    
# #     # Use all together
# #     temp = big_issues_list
# #     temp[0], temp[i] = temp[i], temp[0]
# #     target_big_issue = temp
# #     print(str(i) + "/"+ str(list_len))
# #     print(temp[0])

# #     # K

    
# #         # k = 1
# #         target = big_issues_list[i]
# #         print(str(i+1) + "/"+ str(list_len))
# #         print(target)
# #         target_big_issue = [target]

# #         aux = embedding_results[target]
# #         target_big_issue.extend(aux)
# #         print(aux)   
        
#         # Rand
#         target = big_issues_list[i]
#         print(str(i+1) + "/"+ str(list_len))
#         print(target)
#         target_big_issue = [target]
    
        
#         rand_n = 1
#         temp = np.copy(big_issues_list)
#         np.random.shuffle(temp)
        
#         aux = temp[12]
        
#         while aux in embedding_results[target]:
#             temp = np.copy(big_issues_list)
#             np.random.shuffle(temp)
#             aux = temp[12]

        
#         target_big_issue.append(aux)
#         print(aux)  
        

#         # valid_users, criterion = user_filter(users, target_list, 200)
#         unbalanced_xTr, unbalanced_yTr = get_training_set(valid_users, target_big_issue, big_issues_list, criterion, language_model)

#         xTr = []
#         yTr = []

#         number_of_0 = 0
#         number_of_1 = 0

#         # Find the smaller set
#         for i in range(len(unbalanced_xTr)):
#             if unbalanced_yTr[i][0] == 0:
#                     number_of_0 += 1
#             if unbalanced_yTr[i][0] == 1:
#                     number_of_1 += 1

#         if number_of_0 > number_of_1:
#             number_each = number_of_1
#         else:
#             number_each = number_of_0

#         number_of_0 = 0
#         number_of_1 = 0

#         for i in range(len(unbalanced_xTr)):
#             if unbalanced_yTr[i][0] == 0:
#                 if number_of_0 < number_each:
#                     xTr.append(unbalanced_xTr[i])
#                     yTr.append(unbalanced_yTr[i])
#                     number_of_0 += 1
#             if unbalanced_yTr[i][0] == 1:
#                 if number_of_1 < number_each:
#                     xTr.append(unbalanced_xTr[i])
#                     yTr.append(unbalanced_yTr[i])
#                     number_of_1 += 1

#     #     print(len(xTr), len(yTr))
#     #     print(number_of_0, number_of_1)


#         mask = np.arange(2 * number_each)
#         np.random.shuffle(mask)

#         temp_xTr = []
#         temp_yTr = []

#         for pick in mask:
#             temp_xTr.append(xTr[pick])
#             temp_yTr.append(yTr[pick])

#         xTr = temp_xTr
#         yTr = temp_yTr

#     #     print(len(xTr), len(yTr))
#     #     print(len(xTr[0]), len(yTr[0]))


#         # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#         device = torch.device("cpu")

#         m = model(len(xTr[0]), 64, 2, len(target_big_issue)).to(device)
#         optimizer = optim.SGD(m.parameters(), lr=0.1)

#         training_x = xTr[0:int(0.8 * len(xTr))]
#         training_y = yTr[0:int(0.8 * len(yTr))]
#         test_x = xTr[int(0.8 * len(xTr)):]
#         test_y = yTr[int(0.8 * len(yTr)):]

#         for epoch in (range(10)):
#             m.train()
# #             print('training')
#             start_train = time.time()
#             total_loss = 0

#             total_predictions = 0
#             correct = 0

#             for i in range(int(len(training_x)/10)):
#                 # print(torch.cuda.current_device())
#                 predictions = None
#                 gold_outputs = None
#                 loss = 0
#                 optimizer.zero_grad()

#                 for j in range(10):
#                     loss = 0

# #                     if torch.cuda.is_available():
# #                         gold_output = torch.cuda.FloatTensor(training_y[i + j], device=device)
# #                         input_seq = torch.cuda.FloatTensor(training_x[i + j], device=device)
# #                     else:
#                     gold_output = torch.FloatTensor(training_y[i + j], device=device)
#                     input_seq = torch.FloatTensor(training_x[i + j], device=device)

#                     prediction_vec, prediction = m(input_seq)

#                     total_predictions += 1
#                     # gold_outputs = []
#                     if prediction[0:2] == training_y[i + j][0:2]:
#                         correct += 1
#                     if predictions is None:
#                         predictions = [prediction_vec]
#                         gold_outputs = [gold_output]
#                     else:
#                         predictions.append(prediction_vec)
#                         gold_outputs.append(gold_output)
#                     #
#                     # for lx in range(len(gold_output)):
#                     #     tmp_loss = m.compute_loss(prediction[lx], gold_output[lx])
#                     #     loss += tmp_loss

#                 # print(torch.stack(gold_outputs))
#                 # print(torch.stack(predictions))
#                 loss = m.compute_loss(torch.stack(predictions), torch.stack(gold_outputs).squeeze())
#                 # loss = loss/len(target_big_issue)
#                 loss.backward()
#                 optimizer.step()
#                 total_loss += loss.item()
#             accuracy = correct / total_predictions
# #             print("training time: {} for epoch {}".format(time.time() - start_train, epoch))
# #             print('total loss:{}'.format(total_loss))
# #             print('training accuracy:{}'.format(accuracy))

#         m.eval()
#         predictions = 0
#         correct = 0
#         baseline = 0

#         max_score = 0
#         for i in range(int(len(test_x))):
# #             if torch.cuda.is_available():
# #                 gold_output = torch.cuda.FloatTensor(test_y[i], device=device)
# #                 input_seq = torch.cuda.FloatTensor(test_x[i], device=device)
# #             else:
#             gold_output = torch.tensor(test_y[i], device=device)
#             input_seq = torch.FloatTensor(test_x[i], device=device)

#             _, prediction = m(input_seq)

#             # print("prediction", prediction)
#             # print("test_y", test_y[i])

#             if test_y[i][0:2] == prediction[0:2]:
#                 correct += 1
#             if test_y[i][0:2] == [0.0, 1.0]:
#                 baseline += 1
#             predictions += 1

#         accuracy = correct / predictions
#         acc_collection[target] = accuracy
#         # baseline = baseline / predictions
#         assert 0 <= accuracy <= 1
#         print('Accuracy:{}'.format(accuracy))
#         # print('Baseline:{}'.format(baseline))

#     print(acc_collection)
#     coll.append(acc_collection)

1/49
Abortion
Federal Reserve
Accuracy:0.6103286384976526
2/49
Affirmative Action
Free Trade
Accuracy:0.7371794871794872
3/49
Animal Rights
Environmental Protection
Accuracy:0.6390977443609023
4/49
Barack Obama
War on Terror
Accuracy:0.7542372881355932
5/49
Border Fence
Torture
Accuracy:0.5138461538461538
6/49
Capitalism
Capitalism
Accuracy:0.7037037037037037
7/49
Civil Unions
Federal Reserve
Accuracy:0.5
8/49
Death Penalty
Animal Rights
Accuracy:0.6582278481012658
9/49
Drug Legalization
Social Security
Accuracy:0.6396226415094339
10/49
Electoral College
War in Afghanistan
Accuracy:0.6043956043956044
11/49
Environmental Protection
Civil Unions
Accuracy:0.6530612244897959
12/49
Estate Tax
Military Intervention
Accuracy:0.6449704142011834
13/49
European Union
Euthanasia
Accuracy:0.6127450980392157
14/49
Euthanasia
Social Security
Accuracy:0.6101083032490975
15/49
Federal Reserve
Gun Rights
Accuracy:0.6435185185185185
16/49
Flat Tax
United Nations
Accuracy:0.5711009174311926
17/49
Free Tr